# UNIFICANDO BASES DE DADOS

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
options = Options()
options.add_argument("--start-maximized")
import warnings
warnings.filterwarnings('ignore')

In [22]:
barreiro = pd.read_csv("links_barreiro.csv")
pampulha = pd.read_csv("links_pampulha.csv")
venda_nova = pd.read_csv("links_venda-nova.csv")
zona_centro_sul = pd.read_csv("links_zona-centro-sul.csv")
zona_leste = pd.read_csv("links_zona-leste.csv")
zona_nordeste = pd.read_csv("links_zona-nordeste.csv")
zona_noroeste = pd.read_csv("links_zona-noroeste.csv")
zona_norte = pd.read_csv("links_zona-norte.csv")
zona_oeste = pd.read_csv("links_zona-oeste.csv")

In [23]:
barreiro["REGION"] = "barreiro"
pampulha["REGION"] = "pampulha"
venda_nova["REGION"] = "venda_nova"
zona_centro_sul["REGION"] = "zona_centro_sul"
zona_leste["REGION"] = "zona_leste"
zona_nordeste["REGION"] = "zona_nordeste"
zona_noroeste["REGION"] = "zona_noroeste"
zona_norte["REGION"] = "zona_norte"
zona_oeste["REGION"] = "zona_oeste"

frames = [barreiro, pampulha, venda_nova, zona_centro_sul, zona_leste, zona_nordeste, zona_noroeste, zona_norte, zona_oeste]

links_unificados = pd.concat(frames, ignore_index=True)

# Para salvar o DataFrame unificado em um arquivo csv
links_unificados.to_csv('links_unificados.csv', index=False)


In [24]:
frames = [barreiro, pampulha, venda_nova, zona_centro_sul, zona_leste, zona_nordeste, zona_noroeste, zona_norte, zona_oeste]

links_unificados = pd.concat(frames, ignore_index=True)

In [5]:
duplicate_links = links_unificados[links_unificados.duplicated(['link'])]
len(duplicate_links)


NameError: name 'links_unificados' is not defined

In [57]:
links_unificados.drop_duplicates(subset='link', keep="first", inplace=True)

In [58]:
links_unificados.head()

,titulo,link,date,REGION
0,Apartamento 03 quartos,https://mg.olx.com.br/belo-horizonte-e-regiao/...,2023-08-04 11:32:26.433154,barreiro
2,"Apartamento à venda - Barreiro, 2 quartos - Be...",https://mg.olx.com.br/belo-horizonte-e-regiao/...,2023-08-04 11:32:26.433154,barreiro
4,"Apartamento 02 quartos, 01 vaga à venda no Bai...",https://mg.olx.com.br/belo-horizonte-e-regiao/...,2023-08-04 11:32:26.433154,barreiro
6,Apartamento 03 quartos suite 02 vagas a venda ...,https://mg.olx.com.br/belo-horizonte-e-regiao/...,2023-08-04 11:32:26.433154,barreiro
8,Cobertura para Venda no Bairro Nova Cachoeirinha,https://mg.olx.com.br/belo-horizonte-e-regiao/...,2023-08-04 11:32:26.433154,barreiro


In [59]:
len(links_unificados)

21131

In [60]:
links_unificados['REGION'].value_counts()

zona_oeste         2503
pampulha           2502
zona_leste         2501
venda_nova         2500
zona_nordeste      2500
zona_centro_sul    2499
zona_noroeste      2499
zona_norte         1995
barreiro           1632
Name: REGION, dtype: int64

In [6]:
# Read the CSV into a DataFrame
df_links = pd.read_csv('links_unificados.csv')

# Drop duplicate links
df_links.drop_duplicates(subset='link', inplace=True)

# Save the updated DataFrame back to the CSV
df_links.to_csv('links_unificados.csv', index=False)

##### Dados equilibrados!

### Criando o algoritmo de scrape de informações do apartamento

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
options = Options()
options.add_argument("--start-maximized")
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import re
from tqdm import tqdm



def fetch_data_from_ad(driver, url):
    driver.get(url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1[class="ad__sc-45jt43-0 htAiPK sc-hSdWYo bYQcLm"]')))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    def get_text_by_index(elements, index, default='N/A'):
        return elements[index].text if len(elements) > index else default
    
    all_spans = soup.find_all('span', {'class': 'ad__sc-1f2ug0x-1 cpGpXB sc-hSdWYo gwYTWo'})
    details = soup.find('span', text=re.compile('Detalhes do imóvel'))
    rooms_elem = soup.find('span', text='Quartos')
    rooms_no = rooms_elem.find_next_sibling('div').a.text if rooms_elem and rooms_elem.find_next_sibling('div') else 'N/A'

    data = {
        'TITLE': soup.find('h1', {'class': 'ad__sc-45jt43-0 htAiPK sc-hSdWYo bYQcLm'}).text,
        'LINK': url,
        'DATE_SCRAPE': datetime.today().strftime('%Y-%m-%d'),
        'CEP': get_text_by_index(all_spans, 0),
        'CITY': get_text_by_index(all_spans, 1),
        'NEIGHBORHOOD': get_text_by_index(all_spans, 2),
        'CONDO': get_text_by_index(all_spans, 4),
        'TAX': get_text_by_index(all_spans, 5),
        'AREA': get_text_by_index(all_spans, 6),
        'ROOMS_NO': rooms_no,
        'BATH_NO': get_text_by_index(all_spans, 7),
        'PARKING_SPOTS': get_text_by_index(all_spans, 8),
        'APARTMENT_DETAILS': details.find_next_sibling().text if details and details.find_next_sibling() else 'N/A',
        'PRICE': int(get_text_by_index(soup.find_all('h2', {'class': 'ad__sc-12l420o-1 haeKsn sc-hSdWYo grMlBs'}), 1).replace('.', ''))
    }

    return data

df_detailed = pd.DataFrame()

df_links = pd.read_csv('links_unificados.csv')

driver = webdriver.Chrome(options=options)
total_links = len(df_links)
for i, row in tqdm(enumerate(df_links.iterrows()), total=total_links, desc="Scraping"):
    print(f"Scraping link {i+1}/{total_links}")
    url = row[1]['link']
    region = row[1]['REGION']   # Extracting REGION value
    data = fetch_data_from_ad(driver, url)
    data['REGION'] = region     # Adding REGION to the data dictionary
    df_detailed = df_detailed.append(data, ignore_index=True)
    df_detailed = df_detailed[['TITLE', 'LINK', 'DATE_SCRAPE', 'CITY', 'CEP', 'NEIGHBORHOOD', 'CONDO', 'TAX', 'AREA', 'ROOMS_NO', 'BATH_NO', 'PARKING_SPOTS', 'APARTMENT_DETAILS', 'PRICE', 'REGION']]
    df_detailed.to_csv('dados_detalhados_olx.csv', index=False)
    time.sleep(3)
    print(df_detailed.head())

driver.quit()


Scraping:   0%|                                                                              | 0/21131 [00:00<?, ?it/s]

Scraping link 1/21131


Scraping:   0%|                                                                  | 1/21131 [00:36<213:54:21, 36.44s/it]

                     TITLE                                               LINK  \
0  Apartamento 03 quartos   https://mg.olx.com.br/belo-horizonte-e-regiao/...   

  DATE_SCRAPE            CITY       CEP      NEIGHBORHOOD   CONDO     TAX  \
0  2023-08-05  Belo Horizonte  30662050  Tirol (Barreiro)  R$ 350  R$ 120   

   AREA ROOMS_NO BATH_NO PARKING_SPOTS  \
0  67m²        3       2             1   

                                   APARTMENT_DETAILS   PRICE    REGION  
0  Área de serviço, Armários na cozinha, Armários...  260000  barreiro  
Scraping link 2/21131


Scraping:   0%|                                                                  | 2/21131 [00:40<103:36:26, 17.65s/it]

...

Scraping:   0%|                                                                   | 7/21131 [01:04<36:57:29,  6.30s/it]

                                               TITLE  \
0                            Apartamento 03 quartos    
1                                              Apto    
2  Apartamento à venda - Barreiro, 2 quartos - Be...   
3  Apartamento à Venda - Palmares, 3 Quartos,  85 m2   
4  Apartamento 02 quartos, 01 vaga à venda no Bai...   

                                                LINK DATE_SCRAPE  \
0  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
1  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
2  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
3  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
4  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   

             CITY       CEP            NEIGHBORHOOD   CONDO       TAX  AREA  \
0  Belo Horizonte  30662050        Tirol (Barreiro)  R$ 350    R$ 120  67m²   
1  Belo Horizonte  30620250  Milionários (Barreiro)  R$ 250      R$ 0  66m²   
2  Belo Horizonte

Scraping:   0%|                                                                   | 8/21131 [01:09<50:53:09,  8.67s/it]

                                               TITLE  \
0                            Apartamento 03 quartos    
1                                              Apto    
2  Apartamento à venda - Barreiro, 2 quartos - Be...   
3  Apartamento à Venda - Palmares, 3 Quartos,  85 m2   
4  Apartamento 02 quartos, 01 vaga à venda no Bai...   

                                                LINK DATE_SCRAPE  \
0  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
1  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
2  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
3  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   
4  https://mg.olx.com.br/belo-horizonte-e-regiao/...  2023-08-05   

             CITY       CEP            NEIGHBORHOOD   CONDO       TAX  AREA  \
0  Belo Horizonte  30662050        Tirol (Barreiro)  R$ 350    R$ 120  67m²   
1  Belo Horizonte  30620250  Milionários (Barreiro)  R$ 250      R$ 0  66m²   
2  Belo Horizonte

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0065A813+48355]
	(No symbol) [0x005EC4B1]
	(No symbol) [0x004F5358]
	(No symbol) [0x004DD293]
	(No symbol) [0x0053E37B]
	(No symbol) [0x0054C473]
	(No symbol) [0x0053A536]
	(No symbol) [0x005182DC]
	(No symbol) [0x005193DD]
	GetHandleVerifier [0x008BAABD+2539405]
	GetHandleVerifier [0x008FA78F+2800735]
	GetHandleVerifier [0x008F456C+2775612]
	GetHandleVerifier [0x006E51E0+616112]
	(No symbol) [0x005F5F8C]
	(No symbol) [0x005F2328]
	(No symbol) [0x005F240B]
	(No symbol) [0x005E4FF7]
	BaseThreadInitThunk [0x766A7D59+25]
	RtlInitializeExceptionChain [0x76F5B79B+107]
	RtlClearBits [0x76F5B71F+191]


# Experimentação

Vamos dividir os dados em 4 subsets diferentes para podermos ganhar tempo, depois concatenamos todos

In [16]:
# Read the CSV file
df_links = pd.read_csv('links_unificados.csv')

# Calculate the number of rows in each segment
rows_in_df = df_links.shape[0]
rows_per_segment = rows_in_df // 4

# Split and save the dataframe to 4 separate CSV files
for i in range(4):
    start_idx = i * rows_per_segment
    if i == 3:  # For the last segment, take all remaining rows
        end_idx = rows_in_df
    else:
        end_idx = start_idx + rows_per_segment

    segment = df_links.iloc[start_idx:end_idx]
    segment.to_csv(f'links_unificados_{i+1}.csv', index=False)

print("Splitting completed!")

Splitting completed!
